In [24]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import re
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from datetime import datetime, timedelta
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import pickle

In [2]:
df = pd.read_csv('last_satis.csv')
df.head()

,Unnamed: 0,Tarix,Mağaza,Kart_nomresi,Transaksiya_id,Məhsul_nomresi,Məhsul_adi,Məhsul sayi,Ümumi satış
0,0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


# EDA

In [3]:
df.shape

(200000, 9)

In [4]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.columns

Index(['unnamed:_0', 'tarix', 'mağaza', 'kart_nomresi', 'transaksiya_id',
       'məhsul_nomresi', 'məhsul_adi', 'məhsul_sayi', 'ümumi_satış'],
      dtype='object')

In [5]:
df.isna().sum()

unnamed:_0        0
tarix             0
mağaza            0
kart_nomresi      0
transaksiya_id    0
məhsul_nomresi    0
məhsul_adi        0
məhsul_sayi       0
ümumi_satış       0
dtype: int64

In [6]:
# ID and index columns are dropped!

df.drop(df.columns[[0, 3, 4]], axis=1, inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tarix           200000 non-null  int64  
 1   mağaza          200000 non-null  int64  
 2   məhsul_nomresi  200000 non-null  int64  
 3   məhsul_adi      200000 non-null  object 
 4   məhsul_sayi     200000 non-null  int64  
 5   ümumi_satış     200000 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 9.2+ MB


In [8]:
# Convert Excel-style date to real date
df['tarix'] = pd.to_datetime(df['tarix'], origin = '1899-01-01', unit='D', errors = 'coerce')

# Removing days from date column

df['quarter'] = 'Q' + df['tarix'].dt.quarter.astype(str) + ' ' + df['tarix'].dt.year.astype(str)


In [9]:
df.head()

,tarix,mağaza,məhsul_nomresi,məhsul_adi,məhsul_sayi,ümumi_satış,quarter
0,2017-10-19,1,5,Natural Chip Compny SeaSalt175g,2,6.0,Q4 2017
1,2018-05-16,1,66,CCs Nacho Cheese 175g,3,6.3,Q2 2018
2,2018-05-22,1,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9,Q2 2018
3,2017-08-19,2,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0,Q3 2017
4,2017-08-20,2,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8,Q3 2017


In [10]:
# check for duplicates if there is, drop

df.loc[df.duplicated()]

,tarix,mağaza,məhsul_nomresi,məhsul_adi,məhsul_sayi,ümumi_satış,quarter
3839,2018-06-30,207,56,Cheezels Cheese Box 125g,2,4.2,Q2 2018
5882,2017-12-20,10,100,Smiths Crinkle Cut Chips Chs&Onion170g,2,5.8,Q4 2017
6960,2017-07-07,39,6,RRD Lime & Pepper 165g,2,6.0,Q3 2017
7869,2018-06-21,62,30,Doritos Corn Chips Cheese Supreme 170g,2,8.8,Q2 2018
8061,2018-02-13,65,65,Old El Paso Salsa Dip Chnky Tom Ht300g,2,10.2,Q1 2018
...,...,...,...,...,...,...,...
199928,2017-11-05,210,44,Thins Chips Light& Tangy 175g,2,6.6,Q4 2017
199930,2018-04-28,210,31,Infzns Crn Crnchers Tangy Gcamole 110g,2,7.6,Q2 2018
199943,2018-04-03,210,71,Twisties Cheese Burger 250g,2,8.6,Q2 2018
199944,2017-09-30,210,71,Twisties Cheese Burger 250g,2,8.6,Q3 2017


In [11]:
# Removing duplicates

#df.drop_duplicates(keep = 'first', inplace = True)

In [12]:
df.shape

(200000, 7)

In [13]:
def split_col(col):
    #Extract numeric weight
    weight_match = re.search(r'(\d+)[A-Za-z]*', col)
    weight = weight_match.group(1) if weight_match else ''
    
    # Remove the first occurrence of number + letters from the string
    col_weight = re.sub(r'\d+[A-Za-z]*', '', col, count=1).strip()
    
    #Split remaining string into words
    words = col_weight.split()
    
    #First word is product name, last word is category
    product_name = words[0] if words else ''
    category = words[-1] if words else ''
    
    
    
    return pd.Series([weight, product_name, category])


# Apply the function for each row
df[['weight', 'product_name', 'category']] = df['məhsul_adi'].apply(split_col)

# Remove original column
df.drop('məhsul_adi', axis = 1, inplace = True)

#Display the result
df.head()

,tarix,mağaza,məhsul_nomresi,məhsul_sayi,ümumi_satış,quarter,weight,product_name,category
0,2017-10-19,1,5,2,6.0,Q4 2017,175,Natural,SeaSalt
1,2018-05-16,1,66,3,6.3,Q2 2018,175,CCs,Cheese
2,2018-05-22,1,61,2,2.9,Q2 2018,170,Smiths,Chicken
3,2017-08-19,2,69,5,15.0,Q3 2017,175,Smiths,S/Cream&Onion
4,2017-08-20,2,108,3,13.8,Q3 2017,150,Kettle,Chili


In [14]:
df.isna().sum()

tarix             0
mağaza            0
məhsul_nomresi    0
məhsul_sayi       0
ümumi_satış       0
quarter           0
weight            0
product_name      0
category          0
dtype: int64

In [15]:
# Extracting data for Power BI
#df.to_csv('sales_pbi.csv', index=False)

# Pipeline

In [19]:
# Selectors
from sklearn.compose import make_column_selector

numeric_feature = make_column_selector(dtype_include='number')
categoric_feature = make_column_selector(dtype_include='object')


# Transformers
numeric_transformer = Pipeline(steps = [
    ('scaler', StandardScaler())
])

categoric_transformer = Pipeline(steps=[
    ('encoder', OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1))
])


# ColumnTransformer

preprocessor = ColumnTransformer(transformers = [
    ('num', numeric_transformer, numeric_feature),
    ('cat', categoric_transformer, categoric_feature)
])


# Final model
model = RandomForestRegressor()

# Final pipeline
pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('model', model)
])

In [20]:
# Split
X = df.drop(columns='ümumi_satış')
y = df['ümumi_satış']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 143)

# Fit
pipeline.fit(X_train, y_train)

# Predict
y_pred = pipeline.predict(X_test)


compare_df = pd.DataFrame({
    'Actual' : y_test,
    'Predicted' : y_pred
})

compare_df.head()

,Actual,Predicted
139841,2.6,2.6
45014,5.4,5.4
152178,5.4,5.4
177187,9.2,9.2
6396,7.4,7.4


In [21]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Adjusted R²

n = X_test.shape[0] # number of samples
p = X_test.shape[1] # number of predictors

adj_r2 = 1 - (1-r2)*(n-1) / (n-p-1)

print(f'mae: {mae:.2f}')
print(f'mse: {mse:.2f}')
print(f'r2: {r2:.2f}')
print(f'adjusted_r2: {adj_r2:.2f}')

mae: 0.00
mse: 0.01
r2: 1.00
adjusted_r2: 1.00


In [22]:
df.head()

,tarix,mağaza,məhsul_nomresi,məhsul_sayi,ümumi_satış,quarter,weight,product_name,category
0,2017-10-19,1,5,2,6.0,Q4 2017,175,Natural,SeaSalt
1,2018-05-16,1,66,3,6.3,Q2 2018,175,CCs,Cheese
2,2018-05-22,1,61,2,2.9,Q2 2018,170,Smiths,Chicken
3,2017-08-19,2,69,5,15.0,Q3 2017,175,Smiths,S/Cream&Onion
4,2017-08-20,2,108,3,13.8,Q3 2017,150,Kettle,Chili


In [27]:
with open("sales_model.pkl", "rb") as f:
    model = pickle.load(f)